In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [40]:
import os
import torch

list_dir=['Angry','Neutral','Surprise','Sad','Happy']
emo_idx={'Angry':0,'Neutral':1,'Surprise':2,'Sad':3,'Happy': 4}

def get_feature():
  flag=0
  features=0
  spklabels=0
  emolabels=0

  for i in range(11,21):
      #input_address='C:/Users/64659/Downloads/Emotional Speech Dataset (ESD)/ESD/'+str(i)+'/'
      # output_address='/content/gdrive/MyDrive/paper4_dataset/ESD_devector/'+str(i)+'/'
      # output_address='/content/gdrive/MyDrive/paper4_dataset/ESD_resnet/'+str(i)+'/'
      output_address='/content/gdrive/MyDrive/paper4_dataset/ESD_ecapa/'+str(i)+'/'
      for j in list_dir:
          output_dir_tp=output_address + j +'/'
          tp_emo=torch.tensor([emo_idx[j]], dtype=torch.int64)
          tp_emo=tp_emo.unsqueeze(0)

          for file in os.listdir(output_dir_tp):
            #if num>10: break
            tp_input_address=output_dir_tp + file

            tp=torch.load(tp_input_address,map_location=torch.device('cpu'))
            tp=tp.unsqueeze(0)

            tp_spk = torch.tensor([int(i-11)], dtype=torch.int64)
            tp_spk=tp_spk.unsqueeze(0)
            #print(tp.shape)
            #print(tp2.shape)

            if flag==0:
              flag=1
              features=tp
              spklabels=tp_spk
              emolabels=tp_emo
            else:
                features=torch.cat((features,tp), axis=0)
                spklabels=torch.cat((spklabels,tp_spk), axis=0)
                emolabels=torch.cat((emolabels,tp_emo), axis=0)
  return features, emolabels,spklabels

feats,emolabels,spklabels=get_feature()
print(feats.shape)
print(emolabels.shape)
print(spklabels.shape)

<ipython-input-40-32c29afa58f5>:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  tp=torch.load(tp_input_address,map_location=torch.device('cpu'))


torch.Size([17501, 192])
torch.Size([17501, 1])
torch.Size([17501, 1])


In [41]:
data=torch.cat((feats,emolabels,spklabels),axis=1)
print(data.shape)

torch.Size([17501, 194])


In [4]:
print(data.shape)

torch.Size([17501, 194])


In [42]:
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split

TEST_SIZE = 0.1
BATCH_SIZE = 32
SEED = 42

# generate indices: instead of the actual data we pass in integers instead
train_indices, test_indices, _, _ = train_test_split(
    range(len(data.detach().numpy())),
    data.detach().numpy()[:,-1],
    stratify=data.detach().numpy()[:,-1],
    test_size=TEST_SIZE,
    random_state=SEED
)

# generate subset based on indices
train_split = Subset(data, train_indices)
test_split = Subset(data, test_indices)

# create batches
train_dataloader_simple = DataLoader(train_split, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader_simple = DataLoader(test_split, batch_size=BATCH_SIZE)

In [23]:
import torch.nn as nn
class Network_VAE(nn.Module):
    def __init__(self, input_dim=256, output_dim=256, num_domains=2, hidden_dim=384,emoclassnum=5,spkclassnum=10):
        super().__init__()
        self.output_dim=output_dim
        layers = []
        layers += [nn.Linear(input_dim, hidden_dim)]
        layers += [nn.ReLU()]
        for _ in range(2):
            layers += [nn.Linear(hidden_dim, hidden_dim)]
            layers += [nn.LayerNorm(hidden_dim)]
        layers +=[nn.Linear(hidden_dim, output_dim * 2)]
        layers +=[nn.LayerNorm(output_dim * 2)]


        self.shared = nn.Sequential(*layers)

        # self.speakerencoder=nn.Sequential(*[nn.Linear(hidden_dim, hidden_dim),
        #                                     nn.LayerNorm(hidden_dim),
        #                                     nn.Linear(hidden_dim, output_dim * 2),
        #                                     nn.LayerNorm(hidden_dim)])

        # self.emotionencoder=nn.Sequential(*[nn.Linear(hidden_dim, hidden_dim),
        #                                     nn.LayerNorm(hidden_dim),
        #                                     nn.Linear(hidden_dim, output_dim * 2),
        #                                     nn.LayerNorm(hidden_dim)])

        self.decoder=nn.Sequential(*[nn.Linear(output_dim, hidden_dim),
                                      nn.ReLU(),
                                      nn.Linear(hidden_dim, input_dim)]
                                   )

        # self.emocls = nn.Sequential(*[nn.Linear(output_dim * 2, emoclassnum)])

        # self.spkcls = nn.Sequential(*[nn.Linear(output_dim * 2, spkclassnum)])



        '''
        self.unshared = nn.ModuleList()
        for _ in range(num_domains):
            self.unshared += [nn.Sequential(nn.Linear(hidden_dim, hidden_dim),
                                            nn.ReLU(),
                                            nn.Linear(hidden_dim, hidden_dim),
                                            nn.ReLU(),
                                            nn.Linear(hidden_dim, style_dim))]
        '''
    def forward(self, x):
        h=self.shared(x)

        # emo_mu_logstd=self.emotionencoder(h)
        # spk_mu_logstd=self.speakerencoder(h)

        # emo_mu, emo_log_std = torch.split(emo_mu_logstd,self.output_dim, dim=1)
        # spk_mu, spk_log_std = torch.split(spk_mu_logstd,self.output_dim, dim=1)

        mu, log_std = torch.split(h,self.output_dim, dim=1)

        z = sample(mu, log_std)

        x_hat = self.decoder(z)

        return {'x_hat': x_hat, 'mu': mu, 'log_std': log_std,'z': z}


    def get_feature(self,x):
        h=self.shared(x)

        # spk_mu_logstd=self.speakerencoder(h)

        return h
    '''
    def get_emo(self,x):
        h=self.shared(x)
        emo_mu, emo_log_std = self.emotionencoder(h)
        dec_in = torch.cat([emo_mu, emo_log_std], dim=1)
        emoresult=self.emocls(dec_in)

    def get_spk(self,x):
        h=self.shared(x)
        spk_mu, spk_log_std = self.speakerencoder(h)
        dec_in = torch.cat([spk_mu, spk_log_std], dim=1)
        spkresult=self.spkcls(dec_in)
    '''
    def loss_fn(self, outputs, x_gt):
        x_hat = outputs['x_hat']
        # emo_mu = outputs['emo_mu']
        # emo_log_std = outputs['emo_log_std']
        # spk_mu = outputs['spk_mu']
        # spk_log_std = outputs['spk_log_std']
        # emoresult=outputs['emocls']
        # spkresult=outputs['spkcls']

        # emo_kl = kl_divergence(emo_mu, emo_log_std)
        # spk_kl = kl_divergence(spk_mu, spk_log_std)

        # mi_loss=mutual_information(sample(emo_mu, emo_log_std),sample(spk_mu, spk_log_std))
        loss_2 = nn.MSELoss()
        nll = 0.5*loss_1(x_hat, x_gt) + 0.5*loss_2(x_hat, x_gt)

        # emo_cls=loss_cls(emoresult, emo_gt)
        # spk_cls=loss_cls(spkresult,spk_gt)

        return nll



In [43]:
device=torch.device('cpu')

In [25]:
# Create train_step()
def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               #loss_fn: torch.nn.Module,
               optimizer:torch.optim.Optimizer,
               device=device):
  # Put the model in train mode
  model.train()

  # Setup train loss and train accuracy values
  train_loss = 0

  # Loop through data loader data batches
  for batch, tp in enumerate(dataloader):
    # Send data to the target device
    X=tp[:,:-2]

    X_gt=X

    #y=y.type(torch.LongTensor)
    X, X_gt = X.to(device), X_gt.to(device)

    # 1. Forward pass
    y_pred = model(X) # output model logits
    #print(y_pred['spkcls'].shape)
    #y_pred = torch.softmax(y_pred,dim=1)
    #print(y_pred,y)
    # 2. Calculate the loss
    #y_pred=y_pred.type(torch.float32)
    nll = model.loss_fn(y_pred, X_gt)
    loss=nll
    train_loss += loss.item()

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backward
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    # Calculate accuracy metric
    # y_pred_spkclass = torch.argmax(torch.softmax(y_pred['spkcls'], dim=1), dim=1)
    # spk_train_acc += (y_pred_spkclass==spk_gt).sum().item()/len(y_pred['spkcls'])

    # y_pred_emoclass = torch.argmax(torch.softmax(y_pred['emocls'], dim=1), dim=1)
    # emo_train_acc += (y_pred_emoclass==emo_gt).sum().item()/len(y_pred['emocls'])

  # Adjust metrics to get average loss and accuracy per batch
  train_loss = train_loss / len(dataloader)
  # spk_train_acc = spk_train_acc / len(dataloader)
  # emo_train_acc = emo_train_acc / len(dataloader)
  return train_loss

In [26]:
# Create a test step
def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              #loss_fn: torch.nn.Module,
              device=device):
  # Put model in eval mode
  model.eval()

  # Setup test loss and test accuracy values
  test_loss  = 0

  # Turn on inference mode
  with torch.inference_mode():
    # Loop through DataLoader batches
    for batch, tp in enumerate(dataloader):
      # Send data to the target device
      X=tp[:,:-2]

      X_gt=X

      #y=y.type(torch.LongTensor)
      X, X_gt = X.to(device), X_gt.to(device)

      # 1. Forward pass
      y_pred = model(X) # output model logits

      #y_pred = torch.softmax(y_pred,dim=1)
      #print(y_pred,y)
      # 2. Calculate the loss
      #y_pred=y_pred.type(torch.float32)
      nll = model.loss_fn(y_pred, X_gt)
      loss=nll
      test_loss += loss.item()

      # Calculate accuracy metric
      # y_pred_spkclass = torch.argmax(torch.softmax(y_pred['spkcls'], dim=1), dim=1)
      # spk_test_acc += (y_pred_spkclass==spk_gt).sum().item()/len(y_pred['spkcls'])

      # y_pred_emoclass = torch.argmax(torch.softmax(y_pred['emocls'], dim=1), dim=1)
      # emo_test_acc += (y_pred_emoclass==emo_gt).sum().item()/len(y_pred['emocls'])

    # Adjust metrics to get average loss and accuracy per batch
    test_loss = test_loss / len(dataloader)

  return test_loss



In [27]:

from tqdm.auto import tqdm

# 1. Create a train function that takes in various model parameters + optimizer + dataloaders + loss function
def train(model: torch.nn.Module,
          train_dataloader,
          test_dataloader,
          optimizer,
          #loss_fn: torch.nn.Module = nn.CrossEntropyLoss(),
          epochs: int = 5,
          device=device):

  # 2. Create empty results dictionary
  results = {"train_loss": [],
             "spk_train_acc": [],
             "emo_train_acc": [],
             "test_loss": [],
             "spk_test_acc": [],
             "emo_test_acc": [],}

  # 3. Loop through training and testing steps for a number of epochs
  for epoch in tqdm(range(epochs)):
    train_loss = train_step(model=model,
                                                          dataloader=train_dataloader,
                                                          #loss_fn=loss_fn,
                                                          optimizer=optimizer,
                                                          device=device)
    test_loss= test_step(model=model,
                                                      dataloader=test_dataloader,
                                                      #loss_fn=loss_fn,
                                                      device=device)

    # 4. Print out what's happening
    print(f"Epoch: {epoch} | Train loss: {train_loss:.4f} | Test loss: {test_loss:.4f}")

    # 5. Update results dictionary
    results["train_loss"].append(train_loss)

  # 6. Return the filled results at the end of the epochs
  return results

In [28]:

# Set random seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Set number of epochs
NUM_EPOCHS = 400

# Recreate an instance of TinyVGG
'''
model_0 = TinyVGG(input_shape=3, # number of color channels of our target images
                  hidden_units=10,
                  output_shape=len(train_data.classes)).to(device)
'''
model_main=Network_VAE(input_dim=192,
                        hidden_dim=64)
model_main.to(device)
# Setup loss function and optimizer

optimizer = torch.optim.Adam(params=model_main.parameters(),
                             lr=0.0001)

# Start the timer
from timeit import default_timer as timer
start_time = timer()

# Train model_0
model_0_results = train(model=model_main,
                        train_dataloader=train_dataloader_simple,
                        test_dataloader=test_dataloader_simple,
                        optimizer=optimizer,
                        #loss_fn=loss_fn(),
                        epochs=NUM_EPOCHS)

# End the timer and print out how long it took
end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")


  0%|          | 0/400 [00:00<?, ?it/s]

Epoch: 0 | Train loss: 539.0059 | Test loss: 529.2979
Epoch: 1 | Train loss: 514.0065 | Test loss: 500.7938
Epoch: 2 | Train loss: 494.7468 | Test loss: 492.8026
Epoch: 3 | Train loss: 488.9831 | Test loss: 487.7838
Epoch: 4 | Train loss: 482.1388 | Test loss: 479.6309
Epoch: 5 | Train loss: 471.7778 | Test loss: 468.0294
Epoch: 6 | Train loss: 459.4836 | Test loss: 455.7639
Epoch: 7 | Train loss: 446.9577 | Test loss: 443.7916
Epoch: 8 | Train loss: 434.6117 | Test loss: 431.6837
Epoch: 9 | Train loss: 422.9335 | Test loss: 420.5024
Epoch: 10 | Train loss: 412.0003 | Test loss: 410.1878
Epoch: 11 | Train loss: 402.1258 | Test loss: 401.1795
Epoch: 12 | Train loss: 393.5039 | Test loss: 392.7429
Epoch: 13 | Train loss: 385.8195 | Test loss: 385.5854
Epoch: 14 | Train loss: 378.8753 | Test loss: 379.0601
Epoch: 15 | Train loss: 372.6448 | Test loss: 373.3309
Epoch: 16 | Train loss: 367.2365 | Test loss: 368.4598
Epoch: 17 | Train loss: 362.2685 | Test loss: 363.7787
Epoch: 18 | Train lo

In [29]:
ans=None
model_main.to("cpu")
model_main.eval()
with torch.inference_mode():
  for batch, tp in enumerate(test_dataloader_simple):
    # print(tp.shape)
    X=tp[:,:-2]
    gt=tp[:,-1]



    X.to(device)
    # print(X.shape)
    txxx=model_main.get_feature(X)
    gt = torch.unsqueeze(gt, dim=-1)
    txxx=torch.cat((txxx,gt), axis=1)
    # print(txxx.shape)
    if ans is None:
      ans=txxx
    else:
      ans=torch.cat((ans,txxx), axis=0)

torch.save(ans, "iemecapa_VAE.pt")

In [7]:
import torch.distributions as D
def sample(mu, logs, std=1.):
    """
    :param mu: [batch, channel]
    :param logs:[batch, channel]
    :param std: sampling std
    :return: sample: [batch, channel]
    """
    assert std > 0.
    eps = torch.normal(torch.zeros_like(mu), torch.ones_like(logs) * std)
    sample = eps * torch.exp(logs) + mu
    return sample

def kl_divergence(mu, log_std):
    """
    :param mu: [B, C]
    :param log_std: [B, C]
    :return:
    """
    post = D.Normal(mu, torch.exp(log_std))
    prior = D.Normal(
        torch.zeros_like(mu, requires_grad=False),
        torch.ones_like(log_std, requires_grad=False))
    kl = D.kl.kl_divergence(post, prior)
    kl = torch.mean(torch.sum(kl, dim=1))
    return kl

def loss_cls(x,y):
    loss_3=nn.CrossEntropyLoss()
    # 1. Forward pass
    x=x.type(torch.float32)
    # 2. Calculate the loss
    loss = loss_3(x, y.long())
    return loss


In [8]:
import torch
import torch.nn as nn

class L1SquaredLoss(nn.Module):
    def __init__(self, reduction='mean'):
        super(L1SquaredLoss, self).__init__()
        self.reduction = reduction

    def forward(self, input, target):
        # Compute the absolute differences
        abs_diff = torch.abs(input - target)
        # Square the differences
        squared_diff = abs_diff ** 2

        if self.reduction == 'mean':
            # Compute the mean of squared differences
            return torch.mean(squared_diff)
        elif self.reduction == 'sum':
            # Compute the sum of squared differences
            return torch.sum(squared_diff)
        else:
            # Return the squared differences as is
            return squared_diff

# Example usage
#input = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
#target = torch.tensor([1.5, 2.5, 3.5])

loss_1 = L1SquaredLoss(reduction='mean')
#loss = loss_fn(input, target)

#print("L1 Squared Loss:", loss.item())

In [9]:
import torch
from torch.distributions import Normal

mean = torch.tensor([0.0, 0.0])
std = torch.tensor([1.0,1.0])
# normal_dist = Normal(mean, std)  # 创建一个均值为0，标准差为1的正态分布
sample = mean.rsample()  # 从正态分布中生成一个样本
print(sample)

AttributeError: 'Tensor' object has no attribute 'rsample'

In [19]:
import torch
import numpy as np


tp1=0
tp2=0
for i in range(1,6):

  # x1=torch.load('iemdvector_sess' + str(i) +'_neu1.pt',map_location=torch.device('cpu'))
  # x2=np.loadtxt('iemdvector_sess' + str(i) +'_neu1.npy')+2*(i-1)
  # x1=torch.load('iemres_sess' + str(i) +'_neu1.pt',map_location=torch.device('cpu'))
  # x2=np.loadtxt('iemres_sess' + str(i) +'_neu1.npy')+2*(i-1)
  x1=torch.load('iemepaca_sess' + str(i) +'_neu1.pt',map_location=torch.device('cpu'))
  x2=np.loadtxt('iemepaca_sess' + str(i) +'_neu1.npy')+2*(i-1)
  print(x2.shape)
  if i==1:
    tp1=x1.detach().numpy()
    tp2=x2
  else:
    tp1=np.concatenate((tp1,x1.detach().numpy()), axis=0)
    tp2=np.concatenate((tp2,x2), axis=0)

print(tp2.shape)
tp3=np.zeros((len(tp2)))

(384,)
(360,)
(319,)
(257,)
(381,)
(1701,)


<ipython-input-19-7e3948380d0c>:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  x1=torch.load('iemepaca_sess' + str(i) +'_neu1.pt',map_location=torch.device('cpu'))


In [3]:
print(tp2)

[1. 1. 1. ... 8. 8. 8.]


In [20]:
lis=['ang2','hap','sad','sur']
num=1
for j in lis:
  for i in range(1,6):

    # x1=torch.load('iemdvector_sess' + str(i) +'_' + j + '.pt',map_location=torch.device('cpu'))
    # x2=np.loadtxt('iemdvector_sess' + str(i) +'_' + j + '.npy')+2*(i-1)
    x1=torch.load('iemecapa_sess' + str(i) +'_' + j + '.pt',map_location=torch.device('cpu'))
    x2=np.loadtxt('iemecapa_sess' + str(i) +'_' + j + '.npy')+2*(i-1)
    x3=np.zeros(len(x1))+num
    tp1=np.concatenate((tp1,x1.detach().numpy()), axis=0)
    tp2=np.concatenate((tp2,x2), axis=0)
    tp3=np.concatenate((tp3,x3), axis=0)
  num+=1
print(tp2.shape)

(3974,)


<ipython-input-20-083127cd0ceb>:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  x1=torch.load('iemecapa_sess' + str(i) +'_' + j + '.pt',map_location=torch.device('cpu'))


In [21]:
tp11=torch.tensor(tp1,dtype=torch.float32)

tp22=torch.from_numpy(tp2)
tp22=torch.tensor(tp22,dtype=torch.float32)
tp33=torch.from_numpy(tp3)
tp33=torch.tensor(tp33,dtype=torch.float32)

test_dataset=torch.cat((tp11,tp33.unsqueeze(-1),tp22.unsqueeze(-1)), axis=1)

<ipython-input-21-1c8b21aba9a6>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tp22=torch.tensor(tp22,dtype=torch.float32)
<ipython-input-21-1c8b21aba9a6>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tp33=torch.tensor(tp33,dtype=torch.float32)


In [22]:
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split

TEST_SIZE = 0.1
BATCH_SIZE = 32
SEED = 42

# generate indices: instead of the actual data we pass in integers instead
train_indices, test_indices, _, _ = train_test_split(
    range(len(test_dataset)),
    test_dataset[:,-1],
    stratify=test_dataset[:,-1],
    test_size=TEST_SIZE,
    random_state=SEED
)

# generate subset based on indices
train_split = Subset(test_dataset, train_indices)
test_split = Subset(test_dataset, test_indices)

# create batches
train_dataloader_simple = DataLoader(train_split, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader_simple = DataLoader(test_split, batch_size=BATCH_SIZE)

Beta-VAE

In [30]:
import torch.nn as nn
class Beta_VAE(nn.Module):
    def __init__(self, input_dim=256, output_dim=256, num_domains=2, hidden_dim=384,emoclassnum=5,spkclassnum=10):
        super().__init__()
        self.output_dim=output_dim
        layers = []
        layers += [nn.Linear(input_dim, hidden_dim)]
        layers += [nn.ReLU()]
        for _ in range(2):
            layers += [nn.Linear(hidden_dim, hidden_dim)]
            layers += [nn.LayerNorm(hidden_dim)]
        layers +=[nn.Linear(hidden_dim, output_dim * 2)]
        layers +=[nn.LayerNorm(output_dim * 2)]


        self.shared = nn.Sequential(*layers)

        # self.speakerencoder=nn.Sequential(*[nn.Linear(hidden_dim, hidden_dim),
        #                                     nn.LayerNorm(hidden_dim),
        #                                     nn.Linear(hidden_dim, output_dim * 2),
        #                                     nn.LayerNorm(hidden_dim)])

        # self.emotionencoder=nn.Sequential(*[nn.Linear(hidden_dim, hidden_dim),
        #                                     nn.LayerNorm(hidden_dim),
        #                                     nn.Linear(hidden_dim, output_dim * 2),
        #                                     nn.LayerNorm(hidden_dim)])

        self.decoder=nn.Sequential(*[nn.Linear(output_dim, hidden_dim),
                                      nn.ReLU(),
                                      nn.Linear(hidden_dim, input_dim)]
                                   )

        # self.emocls = nn.Sequential(*[nn.Linear(output_dim * 2, emoclassnum)])

        # self.spkcls = nn.Sequential(*[nn.Linear(output_dim * 2, spkclassnum)])



        '''
        self.unshared = nn.ModuleList()
        for _ in range(num_domains):
            self.unshared += [nn.Sequential(nn.Linear(hidden_dim, hidden_dim),
                                            nn.ReLU(),
                                            nn.Linear(hidden_dim, hidden_dim),
                                            nn.ReLU(),
                                            nn.Linear(hidden_dim, style_dim))]
        '''
    def forward(self, x):
        h=self.shared(x)

        # emo_mu_logstd=self.emotionencoder(h)
        # spk_mu_logstd=self.speakerencoder(h)

        # emo_mu, emo_log_std = torch.split(emo_mu_logstd,self.output_dim, dim=1)
        # spk_mu, spk_log_std = torch.split(spk_mu_logstd,self.output_dim, dim=1)

        mu, log_std = torch.split(h,self.output_dim, dim=1)

        z = sample(mu, log_std)

        x_hat = self.decoder(z)

        return {'x_hat': x_hat, 'mu': mu, 'log_std': log_std,'z': z}


    def get_feature(self,x):
        h=self.shared(x)

        # spk_mu_logstd=self.speakerencoder(h)

        return h
    '''
    def get_emo(self,x):
        h=self.shared(x)
        emo_mu, emo_log_std = self.emotionencoder(h)
        dec_in = torch.cat([emo_mu, emo_log_std], dim=1)
        emoresult=self.emocls(dec_in)

    def get_spk(self,x):
        h=self.shared(x)
        spk_mu, spk_log_std = self.speakerencoder(h)
        dec_in = torch.cat([spk_mu, spk_log_std], dim=1)
        spkresult=self.spkcls(dec_in)
    '''
    def loss_fn(self, outputs, x_gt):
        x_hat = outputs['x_hat']
        mu = outputs['mu']
        log_std = outputs['log_std']
        # spk_mu = outputs['spk_mu']
        # spk_log_std = outputs['spk_log_std']
        # emoresult=outputs['emocls']
        # spkresult=outputs['spkcls']

        kl = kl_divergence(mu, log_std)
        # spk_kl = kl_divergence(spk_mu, spk_log_std)

        # mi_loss=mutual_information(sample(emo_mu, emo_log_std),sample(spk_mu, spk_log_std))
        loss_2 = nn.MSELoss()
        nll = 0.5*loss_1(x_hat, x_gt) + 0.5*loss_2(x_hat, x_gt)

        # emo_cls=loss_cls(emoresult, emo_gt)
        # spk_cls=loss_cls(spkresult,spk_gt)

        return nll,kl

# Create train_step()
def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               #loss_fn: torch.nn.Module,
               optimizer:torch.optim.Optimizer,
               device=device):
  # Put the model in train mode
  model.train()

  # Setup train loss and train accuracy values
  train_loss = 0

  # Loop through data loader data batches
  for batch, tp in enumerate(dataloader):
    # Send data to the target device
    X=tp[:,:-2]

    X_gt=X

    #y=y.type(torch.LongTensor)
    X, X_gt = X.to(device), X_gt.to(device)

    # 1. Forward pass
    y_pred = model(X) # output model logits
    #print(y_pred['spkcls'].shape)
    #y_pred = torch.softmax(y_pred,dim=1)
    #print(y_pred,y)
    # 2. Calculate the loss
    #y_pred=y_pred.type(torch.float32)
    nll,kl = model.loss_fn(y_pred, X_gt)
    loss=nll + beta * kl
    train_loss += loss.item()

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backward
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    # Calculate accuracy metric
    # y_pred_spkclass = torch.argmax(torch.softmax(y_pred['spkcls'], dim=1), dim=1)
    # spk_train_acc += (y_pred_spkclass==spk_gt).sum().item()/len(y_pred['spkcls'])

    # y_pred_emoclass = torch.argmax(torch.softmax(y_pred['emocls'], dim=1), dim=1)
    # emo_train_acc += (y_pred_emoclass==emo_gt).sum().item()/len(y_pred['emocls'])

  # Adjust metrics to get average loss and accuracy per batch
  train_loss = train_loss / len(dataloader)
  # spk_train_acc = spk_train_acc / len(dataloader)
  # emo_train_acc = emo_train_acc / len(dataloader)
  return train_loss

# Create a test step
def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              #loss_fn: torch.nn.Module,
              device=device):
  # Put model in eval mode
  model.eval()

  # Setup test loss and test accuracy values
  test_loss  = 0

  # Turn on inference mode
  with torch.inference_mode():
    # Loop through DataLoader batches
    for batch, tp in enumerate(dataloader):
      # Send data to the target device
      X=tp[:,:-2]

      X_gt=X

      #y=y.type(torch.LongTensor)
      X, X_gt = X.to(device), X_gt.to(device)

      # 1. Forward pass
      y_pred = model(X) # output model logits

      #y_pred = torch.softmax(y_pred,dim=1)
      #print(y_pred,y)
      # 2. Calculate the loss
      #y_pred=y_pred.type(torch.float32)
      nll,kl = model.loss_fn(y_pred, X_gt)
      loss=nll + beta * kl
      test_loss += loss.item()

      # Calculate accuracy metric
      # y_pred_spkclass = torch.argmax(torch.softmax(y_pred['spkcls'], dim=1), dim=1)
      # spk_test_acc += (y_pred_spkclass==spk_gt).sum().item()/len(y_pred['spkcls'])

      # y_pred_emoclass = torch.argmax(torch.softmax(y_pred['emocls'], dim=1), dim=1)
      # emo_test_acc += (y_pred_emoclass==emo_gt).sum().item()/len(y_pred['emocls'])

    # Adjust metrics to get average loss and accuracy per batch
    test_loss = test_loss / len(dataloader)

  return test_loss



from tqdm.auto import tqdm

# 1. Create a train function that takes in various model parameters + optimizer + dataloaders + loss function
def train(model: torch.nn.Module,
          train_dataloader,
          test_dataloader,
          optimizer,
          #loss_fn: torch.nn.Module = nn.CrossEntropyLoss(),
          epochs: int = 5,
          device=device):

  # 2. Create empty results dictionary
  results = {"train_loss": [],
             "spk_train_acc": [],
             "emo_train_acc": [],
             "test_loss": [],
             "spk_test_acc": [],
             "emo_test_acc": [],}

  # 3. Loop through training and testing steps for a number of epochs
  for epoch in tqdm(range(epochs)):
    train_loss = train_step(model=model,
                            dataloader=train_dataloader,
                            #loss_fn=loss_fn,
                            optimizer=optimizer,
                            device=device)
    test_loss= test_step(model=model,
                        dataloader=test_dataloader,
                        #loss_fn=loss_fn,
                        device=device)

    # 4. Print out what's happening
    print(f"Epoch: {epoch} | Train loss: {train_loss:.4f} | Test loss: {test_loss:.4f}")

    # 5. Update results dictionary
    results["train_loss"].append(train_loss)

  # 6. Return the filled results at the end of the epochs
  return results


In [31]:

# Set random seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Set number of epochs
NUM_EPOCHS = 400
beta=4.0

# Recreate an instance of TinyVGG
'''
model_0 = TinyVGG(input_shape=3, # number of color channels of our target images
                  hidden_units=10,
                  output_shape=len(train_data.classes)).to(device)
'''
model_main=Beta_VAE(input_dim=192,
                        hidden_dim=64)
model_main.to(device)
# Setup loss function and optimizer

optimizer = torch.optim.Adam(params=model_main.parameters(),
                             lr=0.0001)

# Start the timer
from timeit import default_timer as timer
start_time = timer()

# Train model_0
model_0_results = train(model=model_main,
                        train_dataloader=train_dataloader_simple,
                        test_dataloader=test_dataloader_simple,
                        optimizer=optimizer,
                        #loss_fn=loss_fn(),
                        epochs=NUM_EPOCHS)

# End the timer and print out how long it took
end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")


  0%|          | 0/400 [00:00<?, ?it/s]

Epoch: 0 | Train loss: 2346.3972 | Test loss: 1633.1527
Epoch: 1 | Train loss: 1482.7758 | Test loss: 1364.2488
Epoch: 2 | Train loss: 1269.7896 | Test loss: 1180.2846
Epoch: 3 | Train loss: 1106.8935 | Test loss: 1045.0126
Epoch: 4 | Train loss: 999.9118 | Test loss: 964.3182
Epoch: 5 | Train loss: 936.7881 | Test loss: 914.2553
Epoch: 6 | Train loss: 891.8576 | Test loss: 871.7898
Epoch: 7 | Train loss: 849.5869 | Test loss: 830.6195
Epoch: 8 | Train loss: 810.2875 | Test loss: 793.5365
Epoch: 9 | Train loss: 777.6366 | Test loss: 764.2899
Epoch: 10 | Train loss: 750.0001 | Test loss: 739.2205
Epoch: 11 | Train loss: 726.3932 | Test loss: 717.2972
Epoch: 12 | Train loss: 706.8942 | Test loss: 697.8740
Epoch: 13 | Train loss: 686.5570 | Test loss: 678.4509
Epoch: 14 | Train loss: 668.1737 | Test loss: 661.7344
Epoch: 15 | Train loss: 654.0659 | Test loss: 649.1614
Epoch: 16 | Train loss: 643.4988 | Test loss: 640.7759
Epoch: 17 | Train loss: 633.1732 | Test loss: 629.9085
Epoch: 18 | 

In [46]:
ans=None
model_main.to("cpu")
model_main.eval()
with torch.inference_mode():
  for batch, tp in enumerate(test_dataloader_simple):
    # print(tp.shape)
    X=tp[:,:-2]
    gt=tp[:,-1]



    X.to(device)
    # print(X.shape)
    txxx=model_main.get_feature(X)
    gt = torch.unsqueeze(gt, dim=-1)
    txxx=torch.cat((txxx,gt), axis=1)
    # print(txxx.shape)
    if ans is None:
      ans=txxx
    else:
      ans=torch.cat((ans,txxx), axis=0)

# torch.save(ans, "iemrvector_VAE.pt")

In [33]:
torch.save(ans, "iemeapa_betaVAE.pt")

In [47]:
torch.save(ans, "iemeapa_infoVAE2.pt")

INFOVAE

In [44]:
import torch.nn as nn
class info_VAE(nn.Module):
    def __init__(self, input_dim=256, output_dim=256, num_domains=2, hidden_dim=384,emoclassnum=5,spkclassnum=10):
        super().__init__()
        self.output_dim=output_dim
        layers = []
        layers += [nn.Linear(input_dim, hidden_dim)]
        layers += [nn.ReLU()]
        for _ in range(2):
            layers += [nn.Linear(hidden_dim, hidden_dim)]
            layers += [nn.LayerNorm(hidden_dim)]
        layers +=[nn.Linear(hidden_dim, output_dim)]
        layers +=[nn.LayerNorm(output_dim)]


        self.shared = nn.Sequential(*layers)

        # self.speakerencoder=nn.Sequential(*[nn.Linear(hidden_dim, hidden_dim),
        #                                     nn.LayerNorm(hidden_dim),
        #                                     nn.Linear(hidden_dim, output_dim * 2),
        #                                     nn.LayerNorm(hidden_dim)])

        # self.emotionencoder=nn.Sequential(*[nn.Linear(hidden_dim, hidden_dim),
        #                                     nn.LayerNorm(hidden_dim),
        #                                     nn.Linear(hidden_dim, output_dim * 2),
        #                                     nn.LayerNorm(hidden_dim)])

        self.decoder=nn.Sequential(*[nn.Linear(output_dim, hidden_dim),
                                      nn.ReLU(),
                                      nn.Linear(hidden_dim, input_dim)]
                                   )

        # self.emocls = nn.Sequential(*[nn.Linear(output_dim * 2, emoclassnum)])

        # self.spkcls = nn.Sequential(*[nn.Linear(output_dim * 2, spkclassnum)])



        '''
        self.unshared = nn.ModuleList()
        for _ in range(num_domains):
            self.unshared += [nn.Sequential(nn.Linear(hidden_dim, hidden_dim),
                                            nn.ReLU(),
                                            nn.Linear(hidden_dim, hidden_dim),
                                            nn.ReLU(),
                                            nn.Linear(hidden_dim, style_dim))]
        '''
    def forward(self, x):
        h=self.shared(x)

        # emo_mu_logstd=self.emotionencoder(h)
        # spk_mu_logstd=self.speakerencoder(h)

        # emo_mu, emo_log_std = torch.split(emo_mu_logstd,self.output_dim, dim=1)
        # spk_mu, spk_log_std = torch.split(spk_mu_logstd,self.output_dim, dim=1)

        # mu, log_std = torch.split(h,self.output_dim, dim=1)

        # z = sample(mu, log_std)
        z= torch.randn(h.shape[0], h.shape[1])

        x_hat = self.decoder(h)

        return {'x_hat': x_hat, 'z': z, 'h':h}


    def get_feature(self,x):
        h=self.shared(x)

        # spk_mu_logstd=self.speakerencoder(h)

        return h
    '''
    def get_emo(self,x):
        h=self.shared(x)
        emo_mu, emo_log_std = self.emotionencoder(h)
        dec_in = torch.cat([emo_mu, emo_log_std], dim=1)
        emoresult=self.emocls(dec_in)

    def get_spk(self,x):
        h=self.shared(x)
        spk_mu, spk_log_std = self.speakerencoder(h)
        dec_in = torch.cat([spk_mu, spk_log_std], dim=1)
        spkresult=self.spkcls(dec_in)
    '''
    def loss_fn(self, outputs, x_gt):
        x_hat = outputs['x_hat']
        # mu = outputs['mu']
        # log_std = outputs['log_std']
        h=outputs['h']
        z=outputs['z']
        # spk_mu = outputs['spk_mu']
        # spk_log_std = outputs['spk_log_std']
        # emoresult=outputs['emocls']
        # spkresult=outputs['spkcls']

        # kl = kl_divergence(mu, log_std)
        # spk_kl = kl_divergence(spk_mu, spk_log_std)
        mmd = compute_mmd(h, z)
        # mi_loss=mutual_information(sample(emo_mu, emo_log_std),sample(spk_mu, spk_log_std))
        loss_2 = nn.MSELoss()
        nll = 0.5*loss_1(x_hat, x_gt) + 0.5*loss_2(x_hat, x_gt)

        # emo_cls=loss_cls(emoresult, emo_gt)
        # spk_cls=loss_cls(spkresult,spk_gt)

        return nll,mmd

# Create train_step()
def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               #loss_fn: torch.nn.Module,
               optimizer:torch.optim.Optimizer,
               device=device):
  # Put the model in train mode
  model.train()

  # Setup train loss and train accuracy values
  train_loss = 0

  # Loop through data loader data batches
  for batch, tp in enumerate(dataloader):
    # Send data to the target device
    X=tp[:,:-2]

    X_gt=X

    #y=y.type(torch.LongTensor)
    X, X_gt = X.to(device), X_gt.to(device)

    # 1. Forward pass
    y_pred = model(X) # output model logits
    #print(y_pred['spkcls'].shape)
    #y_pred = torch.softmax(y_pred,dim=1)
    #print(y_pred,y)
    # 2. Calculate the loss
    #y_pred=y_pred.type(torch.float32)

    nll,mmd = model.loss_fn(y_pred, X_gt)
    loss=nll + mmd
    train_loss += loss.item()

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backward
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    # Calculate accuracy metric
    # y_pred_spkclass = torch.argmax(torch.softmax(y_pred['spkcls'], dim=1), dim=1)
    # spk_train_acc += (y_pred_spkclass==spk_gt).sum().item()/len(y_pred['spkcls'])

    # y_pred_emoclass = torch.argmax(torch.softmax(y_pred['emocls'], dim=1), dim=1)
    # emo_train_acc += (y_pred_emoclass==emo_gt).sum().item()/len(y_pred['emocls'])

  # Adjust metrics to get average loss and accuracy per batch
  train_loss = train_loss / len(dataloader)
  # spk_train_acc = spk_train_acc / len(dataloader)
  # emo_train_acc = emo_train_acc / len(dataloader)
  return train_loss

# Create a test step
def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              #loss_fn: torch.nn.Module,
              device=device):
  # Put model in eval mode
  model.eval()

  # Setup test loss and test accuracy values
  test_loss  = 0

  # Turn on inference mode
  with torch.inference_mode():
    # Loop through DataLoader batches
    for batch, tp in enumerate(dataloader):
      # Send data to the target device
      X=tp[:,:-2]

      X_gt=X

      #y=y.type(torch.LongTensor)
      X, X_gt = X.to(device), X_gt.to(device)

      # 1. Forward pass
      y_pred = model(X) # output model logits

      #y_pred = torch.softmax(y_pred,dim=1)
      #print(y_pred,y)
      # 2. Calculate the loss
      #y_pred=y_pred.type(torch.float32)
      nll,mmd = model.loss_fn(y_pred, X_gt)
      loss=nll + mmd
      test_loss += loss.item()

      # Calculate accuracy metric
      # y_pred_spkclass = torch.argmax(torch.softmax(y_pred['spkcls'], dim=1), dim=1)
      # spk_test_acc += (y_pred_spkclass==spk_gt).sum().item()/len(y_pred['spkcls'])

      # y_pred_emoclass = torch.argmax(torch.softmax(y_pred['emocls'], dim=1), dim=1)
      # emo_test_acc += (y_pred_emoclass==emo_gt).sum().item()/len(y_pred['emocls'])

    # Adjust metrics to get average loss and accuracy per batch
    test_loss = test_loss / len(dataloader)

  return test_loss



from tqdm.auto import tqdm

# 1. Create a train function that takes in various model parameters + optimizer + dataloaders + loss function
def train(model: torch.nn.Module,
          train_dataloader,
          test_dataloader,
          optimizer,
          #loss_fn: torch.nn.Module = nn.CrossEntropyLoss(),
          epochs: int = 5,
          device=device):

  # 2. Create empty results dictionary
  results = {"train_loss": [],
             "spk_train_acc": [],
             "emo_train_acc": [],
             "test_loss": [],
             "spk_test_acc": [],
             "emo_test_acc": [],}

  # 3. Loop through training and testing steps for a number of epochs
  for epoch in tqdm(range(epochs)):
    train_loss = train_step(model=model,
                            dataloader=train_dataloader,
                            #loss_fn=loss_fn,
                            optimizer=optimizer,
                            device=device)
    test_loss= test_step(model=model,
                        dataloader=test_dataloader,
                        #loss_fn=loss_fn,
                        device=device)

    # 4. Print out what's happening
    print(f"Epoch: {epoch} | Train loss: {train_loss:.4f} | Test loss: {test_loss:.4f}")

    # 5. Update results dictionary
    results["train_loss"].append(train_loss)

  # 6. Return the filled results at the end of the epochs
  return results


In [35]:
def compute_kernel(x, y):
    x_size = x.size(0)
    y_size = y.size(0)
    dim = x.size(1)
    x = x.unsqueeze(1) # (x_size, 1, dim)
    y = y.unsqueeze(0) # (1, y_size, dim)
    tiled_x = x.expand(x_size, y_size, dim)
    tiled_y = y.expand(x_size, y_size, dim)
    kernel_input = (tiled_x - tiled_y).pow(2).mean(2)/float(dim)
    return torch.exp(-kernel_input) # (x_size, y_size)

def compute_mmd(x, y):
    x_kernel = compute_kernel(x, x)
    y_kernel = compute_kernel(y, y)
    x_kernel.to(device), y_kernel.to(device)
    xy_kernel = compute_kernel(x, y)
    mmd = x_kernel.mean() + y_kernel.mean() - 2*xy_kernel.mean()
    return mmd

In [45]:

# Set random seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Set number of epochs
NUM_EPOCHS = 400
beta=4.0

# Recreate an instance of TinyVGG
'''
model_0 = TinyVGG(input_shape=3, # number of color channels of our target images
                  hidden_units=10,
                  output_shape=len(train_data.classes)).to(device)
'''
model_main=info_VAE(input_dim=192,
                        hidden_dim=64)
model_main.to(device)
# Setup loss function and optimizer

optimizer = torch.optim.Adam(params=model_main.parameters(),
                             lr=0.0001)

# Start the timer
from timeit import default_timer as timer
start_time = timer()

# Train model_0
model_0_results = train(model=model_main,
                        train_dataloader=train_dataloader_simple,
                        test_dataloader=test_dataloader_simple,
                        optimizer=optimizer,
                        #loss_fn=loss_fn(),
                        epochs=NUM_EPOCHS)

# End the timer and print out how long it took
end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")


  0%|          | 0/400 [00:00<?, ?it/s]

Epoch: 0 | Train loss: 458.0009 | Test loss: 412.4364
Epoch: 1 | Train loss: 376.3608 | Test loss: 345.2039
Epoch: 2 | Train loss: 320.4447 | Test loss: 302.1810
Epoch: 3 | Train loss: 289.6821 | Test loss: 280.1376
Epoch: 4 | Train loss: 271.3905 | Test loss: 265.3852
Epoch: 5 | Train loss: 260.3933 | Test loss: 257.2403
Epoch: 6 | Train loss: 253.1629 | Test loss: 250.6230
Epoch: 7 | Train loss: 246.4671 | Test loss: 244.1321
Epoch: 8 | Train loss: 240.2764 | Test loss: 238.1471
Epoch: 9 | Train loss: 234.3004 | Test loss: 232.4162
Epoch: 10 | Train loss: 228.5230 | Test loss: 226.7806
Epoch: 11 | Train loss: 222.9209 | Test loss: 221.0444
Epoch: 12 | Train loss: 217.1958 | Test loss: 215.6996
Epoch: 13 | Train loss: 212.1139 | Test loss: 210.7946
Epoch: 14 | Train loss: 207.5037 | Test loss: 206.4530
Epoch: 15 | Train loss: 203.3943 | Test loss: 202.5369
Epoch: 16 | Train loss: 199.5136 | Test loss: 198.8242
Epoch: 17 | Train loss: 195.7700 | Test loss: 195.2823
Epoch: 18 | Train lo